In [1]:
import os
from getpass import getpass

import pandas as pd
import ast
import json
import re
import jieba

import torch

from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace


from langchain_ollama.llms import OllamaLLM

from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/home/vince-ylf/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
llm = OllamaLLM(model='gemma3:4b-it-qat', temperature=1)

template = """你是一位擅長分類文本主題的助手，以下是從 LDA 分類出的四個主題，每個主題最重要的10個關鍵字，請根據每個主題的10個關鍵字，推導出最適合的主題類別:
{content}

===範例:
input:
[(0, '0.010*"NBA" + 0.008*"湖人" + 0.008*"kobe" + 0.007*"籃板" + 0.007*"三分球" + 0.006*"灌籃" + 0.006*"冠軍" + 0.006*"西區" + 0.005*"東區" + 0.005*"美國"'),
(1, '0.010*"MLB" + 0.008*"盜壘" + 0.008*"全壘打" + 0.007*"投手" + 0.007*"打擊率" + 0.007*"觸身球" + 0.005*"滾地球" + 0.005*"三振" + 0.005*"美國" + 0.004*"球場"'),
]

output: 
[(0, '籃球'), (1, '棒球')]
"""

prompt = PromptTemplate.from_template(template)

prompt_output = prompt.invoke({"content": """[(0,
  '0.010*"地震" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"表示" + 0.007*"報導" + 0.006*"立委" + 0.006*"國民黨" + 0.006*"提供" + 0.005*"民進黨" + 0.005*"台灣"'),
 (1,
  '0.010*"表示" + 0.008*"地震" + 0.008*"記者" + 0.007*"報導" + 0.007*"花蓮" + 0.007*"立法院" + 0.005*"立委" + 0.005*"國民黨" + 0.005*"台灣" + 0.004*"民進黨"'),
 (2,
  '0.010*"立委" + 0.008*"表示" + 0.007*"國民黨" + 0.007*"民進黨" + 0.006*"報導" + 0.006*"立法院" + 0.006*"記者" + 0.005*"台灣" + 0.004*"國會" + 0.004*"民眾"'),
 (3,
  '0.011*"台灣" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"民進黨" + 0.007*"表示" + 0.007*"報導" + 0.006*"地震" + 0.005*"國民黨" + 0.005*"立法院" + 0.004*"立委"')]"""})
prompt_output

StringPromptValue(text='你是一位擅長分類文本主題的助手，以下是從 LDA 分類出的四個主題，每個主題最重要的10個關鍵字，請根據每個主題的10個關鍵字，推導出最適合的主題類別:\n[(0,\n  \'0.010*"地震" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"表示" + 0.007*"報導" + 0.006*"立委" + 0.006*"國民黨" + 0.006*"提供" + 0.005*"民進黨" + 0.005*"台灣"\'),\n (1,\n  \'0.010*"表示" + 0.008*"地震" + 0.008*"記者" + 0.007*"報導" + 0.007*"花蓮" + 0.007*"立法院" + 0.005*"立委" + 0.005*"國民黨" + 0.005*"台灣" + 0.004*"民進黨"\'),\n (2,\n  \'0.010*"立委" + 0.008*"表示" + 0.007*"國民黨" + 0.007*"民進黨" + 0.006*"報導" + 0.006*"立法院" + 0.006*"記者" + 0.005*"台灣" + 0.004*"國會" + 0.004*"民眾"\'),\n (3,\n  \'0.011*"台灣" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"民進黨" + 0.007*"表示" + 0.007*"報導" + 0.006*"地震" + 0.005*"國民黨" + 0.005*"立法院" + 0.004*"立委"\')]\n\n===範例:\ninput:\n[(0, \'0.010*"NBA" + 0.008*"湖人" + 0.008*"kobe" + 0.007*"籃板" + 0.007*"三分球" + 0.006*"灌籃" + 0.006*"冠軍" + 0.006*"西區" + 0.005*"東區" + 0.005*"美國"\'),\n(1, \'0.010*"MLB" + 0.008*"盜壘" + 0.008*"全壘打" + 0.007*"投手" + 0.007*"打擊率" + 0.007*"觸身球" + 0.005*"滾地球" + 0.005*"三振" + 0.005*"美國" + 0.004*"球場"\'),\n]\n\noutp

In [3]:
print(prompt_output)
llm_output = llm.invoke(prompt_output)
llm_output

text='你是一位擅長分類文本主題的助手，以下是從 LDA 分類出的四個主題，每個主題最重要的10個關鍵字，請根據每個主題的10個關鍵字，推導出最適合的主題類別:\n[(0,\n  \'0.010*"地震" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"表示" + 0.007*"報導" + 0.006*"立委" + 0.006*"國民黨" + 0.006*"提供" + 0.005*"民進黨" + 0.005*"台灣"\'),\n (1,\n  \'0.010*"表示" + 0.008*"地震" + 0.008*"記者" + 0.007*"報導" + 0.007*"花蓮" + 0.007*"立法院" + 0.005*"立委" + 0.005*"國民黨" + 0.005*"台灣" + 0.004*"民進黨"\'),\n (2,\n  \'0.010*"立委" + 0.008*"表示" + 0.007*"國民黨" + 0.007*"民進黨" + 0.006*"報導" + 0.006*"立法院" + 0.006*"記者" + 0.005*"台灣" + 0.004*"國會" + 0.004*"民眾"\'),\n (3,\n  \'0.011*"台灣" + 0.008*"花蓮" + 0.008*"記者" + 0.007*"民進黨" + 0.007*"表示" + 0.007*"報導" + 0.006*"地震" + 0.005*"國民黨" + 0.005*"立法院" + 0.004*"立委"\')]\n\n===範例:\ninput:\n[(0, \'0.010*"NBA" + 0.008*"湖人" + 0.008*"kobe" + 0.007*"籃板" + 0.007*"三分球" + 0.006*"灌籃" + 0.006*"冠軍" + 0.006*"西區" + 0.005*"東區" + 0.005*"美國"\'),\n(1, \'0.010*"MLB" + 0.008*"盜壘" + 0.008*"全壘打" + 0.007*"投手" + 0.007*"打擊率" + 0.007*"觸身球" + 0.005*"滾地球" + 0.005*"三振" + 0.005*"美國" + 0.004*"球場"\'),\n]\n\noutput: \n[(0, \'籃球\')

'```\n[(0, \'地震災難\'), (1, \'花蓮地震\'), (2, \'台灣政局\'), (3, \'台灣地震\')]\n```\n\n**解釋：**\n\n*   **主題 0:**  關鍵字 "地震", "花蓮", "記者", "報導", "立法院", "立委", "台灣" 表明主要討論的是花蓮地震事件以及相關的報導和政治反應。因此，分類為“地震災難”或“花蓮地震”更合適。\n*   **主題 1:**  關鍵字 "台灣", "立法院", "立委", "國民黨", "台灣" 表明與台灣政治有關，可能涉及立法院的討論或國民黨的立場。因此，分類為“台灣政局”或“台灣地震”更合適。\n*   **主題 2:**  關鍵字 "立委", "報導", "國民黨", "台灣", "國會", "民眾" 表明與台灣政治和相關議題的報導有關，可能關注民意和政治人物。因此，分類為“台灣政局”或“台灣地震”更合適。\n*   **主題 3:**  關鍵字 “台灣”, “花蓮”, “地震”, “國民黨”, “立法院”, “立委” 表明與台灣地震有關的政治狀況。因此，分類為“台灣地震”更合適。\n\n希望這個回答對您有幫助！\n'

In [4]:
udn = pd.read_csv("student_2_5_8.csv")  # 匯資料
udn.head(3)

,artTitle,artContent
0,郭子乾趕高鐵巧遇王世堅 網譏「想看韓院長表示....」,郭子乾（左）和王世堅昨在高鐵站巧遇，開心拍下同框合影。（翻攝自郭子乾臉書）\n〔記者林南谷／...
1,國民黨智庫籲新政府解除不合時宜兩岸禁令 恢復正常往來,2024/05/14 15:49\n\n\n 國民黨智庫表示，國民黨將在智庫國安組召集人陳永...
2,漲電價要立法院同意！藍白再聯手表決通過「電業法」逕付二讀,2024/05/03 11:54\n\n\n 立法院議場上演藍綠表決攻防，國民黨團高喊凍漲電...


In [5]:
# parser = StrOutputParser()

# food_template = """你是一位擅長分類文本主題的助手，請幫我將該文本分類為 '災害應變', '花蓮地震', '政治聲明','台灣地震' 四種，
# 文章只能分類成這四種繁體中文的類別名稱，輸出必須是 string，不能有任何標點符號或markdown。

# 以下為一些範例：
# ```
# 範例1
# 句子： 漲電價要立法院同意！藍白再聯手表決通過「電業法」逕付二讀
# 結果： "政治聲明"
# ""

# 範例2
# 句子：台北市長蔣萬安透露，花蓮地震後，民眾通報建物需勘查件數逾1800件。（資料照）〔記者何玉華／台北報導〕花蓮強震後餘震不斷，台北市長蔣萬安表示，從花蓮大地震發生迄今，民眾擔心建物通報達1800多件，都發局、區公所、還有三大技師公會，全面協助民眾做建物的檢視，目前平均每天都還有約100通的通報進線。建管處補充說明，花蓮地震後通報要勘查共1832件，已勘查完畢924件，確認紅單2件，黃單17件，持續勘查中。蔣萬安表示，花蓮大地震發生迄今，民眾通報建物受損達1800多件，都發局、區公所、還有三大技師公會全面協助民眾做建物檢視，也針對紅、黃單建物全面再檢視一次，目前已經完成花蓮大地震的紅、黃單建物，即刻正在進行921跟331的建物，目前平均每天都還有約100通的通報進線。蔣萬安也提到，昨天半夜2點半地震發生後，他起來好多次，「第一時間太太動作比我快，先撲著三寶，我撲著我太太」，最重要的是現在還有各個餘震，除了提醒民眾做好各項的準備，也請都發局注意建築物外牆的外掛物，不管是廣告物還是施工工地的機具、高空作業的地方等，都要留意加強穩固。蔣萬安說，要持續加強各項防災的宣導，讓防災的思維導入到校園還有日常民眾生活的體系當中。對於危險建築更新也要加速，優先請教育局加速學校校舍改建的期程，全市各項建物也要加快相關的更新作業，不只是都更8箭加大都更政策的力道，也需要跟中央來協調，如之前受花蓮地震影響最嚴重的南機場整宅，希望跟中央協調到中繼安置的部分，加速更新進行。
# 結果："花蓮地震"

# ===文本:
# {text}
# """

# def choose_category(row):
#     """選擇類別"""
#     food_prompt = PromptTemplate(
#         template=food_template,
#         input_variables=["query"],
#         # 將 format_instructions 也輸入到 prompt templete 中
#         partial_variables={"text": row['artContent']},
#     )
#     chain = food_prompt | llm | parser
#     return chain.invoke({"query": row['artContent']})

# udn['category'] = udn.apply(lambda row: choose_category, axis=1)

In [ ]:
import pandas as pd
from langchain import PromptTemplate

# 讀取資料
udn = pd.read_csv("student_2_5_8.csv")

# 定義 parser
parser = StrOutputParser()

# 定義 prompt template，注意 input_variables 要跟模板裡用到的 {text} 一致
food_template = """你是一位擅長分類文本主題的助手，請幫我將該文本分類為 '地震災難', '台灣政局', '政治聲明','台灣地震' 四種，
文章只能分類成這四種繁體中文的類別名稱中的一種，輸出必須是 string，不能有任何標點符號或markdown。

===文本:
{text}
"""

food_prompt = PromptTemplate(
    template=food_template,
    input_variables=["text"],
)

def choose_category(row):
    # 透過 partial 把當前這一列的 artContent 填入 prompt
    chain = food_prompt.partial(text=row["artContent"]) | llm | parser
    # invoke 時不需要再傳入任何額外參數
    return chain.invoke({})

# 正確呼叫 apply，把 choose_category 函式作用到每一列
udn["category"] = udn.apply(choose_category, axis=1)

# 看一下結果
print(udn[["artContent", "category"]].head(5))


                                          artContent category
0  郭子乾（左）和王世堅昨在高鐵站巧遇，開心拍下同框合影。（翻攝自郭子乾臉書）\n〔記者林南谷／...   政治聲明\n
1  2024/05/14 15:49\n\n\n 國民黨智庫表示，國民黨將在智庫國安組召集人陳永...   政治聲明\n
2  2024/05/03 11:54\n\n\n 立法院議場上演藍綠表決攻防，國民黨團高喊凍漲電...   政治聲明\n
3  2024/05/03 11:07\n\n\n 國民黨智庫3日召開「520新政府 新期待 別讓...   政治聲明\n
4  2024/05/14 12:17\n\n\n 立法院長韓國瑜敲下議事槌，三讀通過姓名條例部分...   政治聲明\n


In [7]:
udn

,artTitle,artContent,category
0,郭子乾趕高鐵巧遇王世堅 網譏「想看韓院長表示....」,郭子乾（左）和王世堅昨在高鐵站巧遇，開心拍下同框合影。（翻攝自郭子乾臉書）\n〔記者林南谷／...,政治聲明\n
1,國民黨智庫籲新政府解除不合時宜兩岸禁令 恢復正常往來,2024/05/14 15:49\n\n\n 國民黨智庫表示，國民黨將在智庫國安組召集人陳永...,政治聲明\n
2,漲電價要立法院同意！藍白再聯手表決通過「電業法」逕付二讀,2024/05/03 11:54\n\n\n 立法院議場上演藍綠表決攻防，國民黨團高喊凍漲電...,政治聲明\n
3,徐巧芯丈夫被控過去是挺朱網軍 朱立倫：外界不應牽拖,2024/05/03 11:07\n\n\n 國民黨智庫3日召開「520新政府 新期待 別讓...,政治聲明\n
4,「姓名條例修正草案」三讀通過 原住民族可於身分證單列族名,2024/05/14 12:17\n\n\n 立法院長韓國瑜敲下議事槌，三讀通過姓名條例部分...,政治聲明\n
...,...,...,...
744,台積電大漲 市值重返20兆元大關,台積電市值重返20兆元大關。（記者洪友芳攝）\n〔記者洪友芳／新竹報導〕晶圓代工龍頭台積電（...,政治聲明\n
745,邱求慧︰輝達AI研發中心 來台設立,經濟部技術司長邱求慧。（記者于安亭攝）「官我什麼事」專訪透露 台灣擁硬體、基建、專業人才 吸...,台灣地震\n
746,別怕！美銀：這數字出現前 美股牛市就不會結束,首次上稿 00:34更新時間 05:59\n美銀表示，美股還沒漲完，在10年期公債殖利率上升...,政治聲明\n
747,《晶片戰爭》作者：美晶片法目前成功讓人意外,近期美國政府分別向英特爾（Intel）、台積電（2330）、三星（Samsung）與美光（M...,災害應變\n花蓮地震\n台灣地震\n政治聲明
